In [13]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader


In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])


In [15]:
train_data = datasets.ImageFolder(
    root=r"chestctscan\Data\train",
    transform=transform
)

test_data = datasets.ImageFolder(
    root=r"chestctscan\Data\test",
    transform=transform
)


In [16]:
train_loader = DataLoader(
    train_data, batch_size=8, shuffle=True, num_workers=0, pin_memory=True
)
test_loader = DataLoader(
    test_data, batch_size=8, num_workers=0, pin_memory=True
)

In [17]:
class Chest(nn.Module):
  def __init__(self):
    super().__init__()
    self.base = models.resnet18(pretrained=True)
    self.base.fc = nn.Linear(self.base.fc.in_features,4)
  def forward(self,x):
    return self.base(x)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Chest().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [19]:
!pip install tqdm


In [20]:
from tqdm import tqdm
def train():
  model.train()
  total_loss = 0
  for images,labels in tqdm(train_loader, desc="Training", leave=False):
    images,labels = images.to(device),labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = loss_fn(output,labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(train_loader)


In [21]:
def evaluate():
  model.eval()
  correct,total = 0,0
  with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
      images,labels = images.to(device),labels.to(device)
      preds = torch.argmax(model(images),dim=1)
      correct += (preds==labels).sum().item()
      total += labels.size(0)
  return correct/total

In [23]:
for epoch in tqdm(range(1, 30), desc="Epochs"):
  avg_loss = train()
  acc = evaluate()
  print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}")

Epochs:   3%|▎         | 1/29 [00:08<04:07,  8.84s/it]

Epoch: 1, Loss: 0.3242, Accuracy: 0.8000


Epochs:   7%|▋         | 2/29 [00:17<03:51,  8.56s/it]

Epoch: 2, Loss: 0.2439, Accuracy: 0.7778


Epochs:  10%|█         | 3/29 [00:25<03:38,  8.42s/it]

...

Epoch: 27, Loss: 0.0372, Accuracy: 0.9365


Epochs:  97%|█████████▋| 28/29 [03:50<00:08,  8.23s/it]

Epoch: 28, Loss: 0.0389, Accuracy: 0.8413


Epochs: 100%|██████████| 29/29 [03:58<00:00,  8.22s/it]

Epoch: 29, Loss: 0.0358, Accuracy: 0.8889


In [24]:
torch.save(model.state_dict(), "chestmri.pth")

In [25]:
print("Model saved as chestmri.pth")

Model saved as chestmri.pth


<function list.count(value, /)>